In [14]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os

In [15]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

In [16]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
        print(obj['Size'])
else:
    print("No objects found in the bucket.")

2023-06-01.csv
505500
2023-06-02.csv
509187
STAC_LiDAR/
0
STAC_LiDAR/PointClouds/
0
STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz
23236093
STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz
140355729
STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz
50122462
STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz
95552259
STAC_LiDAR/PointClouds/bc_092o018_4_1_4_xyes_12_utm10_2018.laz
336226672
STAC_LiDAR/PointClouds/bc_092o018_4_3_1_xyes_12_utm10_2018.laz
315611463
STAC_LiDAR/PointClouds/bc_092o018_4_3_2_xyes_12_utm10_2018.laz
354790466
STAC_LiDAR/PointClouds/bc_092o018_4_3_3_xyes_12_utm10_2018.laz
312537985
STAC_LiDAR/PointClouds/bc_092o018_4_3_4_xyes_12_utm10_2018.laz
332078905
_$folder$
0
geotest/
0
geotest/_$folder$
0
geotest/cog/082E01_rgb.tif
1739342484
geotest/cog/_$folder$
0
geotest/cog/lzw2_082E01_rgb.tif
2135529718
geotest/cog/lzw_082E01_rgb.tif
2135588664
geotest/veg_comp_layer_r1_poly.parquet
11654788107


In [32]:
# specify the folder path (in the current working directory) for your object 
# if the folder doesn't exist, create it
save_path = os.path.join(os.getcwd(), r'junk')
if not os.path.exists(save_path):
    os.makedirs(save_path)

# create new text file within the folder
dest_file_path = os.path.join(save_path, r'test.txt')

# write data to the new text file
with open(dest_file_path, 'w') as fh:
    fh.write('example example example\n')

In [ ]:
# put the new text file into your object storage bucket
s3_client.put_object(Bucket=bucket, Key=dest_file_path)

In [34]:
# define the function for creating presigned URL 
# running the function will return a shareable URL in the print output
def create_presigned_url(
    bucket_name = bucket,
    object_name = dest_file_path,
    expiration=180,
):

    try:
        response = s3_client.generate_presigned_url(
            "get_object",
            Params={"Bucket": bucket_name, "Key": object_name},
            ExpiresIn=expiration,
            )
        print(response)

    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [ ]:
# run the presigned URL function with the given parameters
create_presigned_url(bucket, dest_file_path)

In [ ]:
# delete the remote object
s3_client.delete_object(
    Bucket=bucket, 
    Key=dest_file_path
    )

In [1]:
# add folder to S3 bucket 
def uploadDirectory(path,bucketname):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucketname,file)